# Лабораторная работа 4

* Убедиться в нестабильности одиночного дерева на своих данных
* Отобрать самые важные признаки случайным лесом, сравнить с отбором признаков линейным методом  c L1-регуляризацией
* Сравнить качество работы случайного леса без кросс-валидации с кросс-валидации
* Сравнить качество работы и время обучения леса с градиентным бустингом над решающими деревьями, подобрав для каждого оптимальные параметры. Градиентный бустинг обучать на видеокарте.

In [1]:
# загрузим данные

#импорт необходимых модулей
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pylab import rcParams

# результат по выживани.
trainSurvivedPath = '/home/alex/Downloads/bmstuML/dataMl/trainSurvived.plk'
testSurvivedPath = '/home/alex/Downloads/bmstuML/dataMl/testSurvived.plk'

y_train = pd.read_pickle(trainSurvivedPath)
y_test = pd.read_pickle(testSurvivedPath)


# подготовленные данные из ЛР 2
trainDataPath = '/home/alex/Downloads/bmstuML/dataMl/trainData.plk'
testDataPath = '/home/alex/Downloads/bmstuML/dataMl/testData.plk'

X_train = pd.read_pickle(trainDataPath)
X_test = pd.read_pickle(testDataPath)


X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714 entries, 0 to 713
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Sex     714 non-null    float64
 1   Age     714 non-null    float64
 2   SibSp   714 non-null    float64
 3   Parch   714 non-null    float64
 4   Fare    714 non-null    float64
 5   is_S    714 non-null    float64
 6   is_C    714 non-null    float64
 7   is_Q    714 non-null    float64
dtypes: float64(8)
memory usage: 44.8 KB


### Нестабильность одиночного дерева

In [6]:
#необходимый импорт

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import pylab as plt
import seaborn as sns
from matplotlib import pyplot as plt

In [7]:
# def get_grid(data, eps=0.01):
#     x_min, x_max = data[:,0].min() - 1, data[:,0].max() + 1
#     y_min, y_max = data[:,1].min() - 1, data[:,1].max() + 1
#     return np.meshgrid(np.arange(x_min, x_max, eps),
#                       np.arange(y_min, y_max, eps))


# plt.rcParams['figure.figsize'] = (10,8)

In [55]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score

#max_features=3
clf_tree = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=15)

clf_tree.fit(X_train, y_train)

predicted = clf_tree.predict(X_test)

prec_score = precision_score(y_test, predicted)

print('precision_score:', prec_score)



precision_score: 0.8807339449541285


In [26]:
X_test.head(1)

,Sex,Age,SibSp,Parch,Fare,is_S,is_C,is_Q
0,1.0,0.452723,0.0,0.0,0.015282,0.0,0.0,1.0


In [43]:
from ipywidgets import Image
from io import StringIO
import pydotplus
from sklearn.tree import export_graphviz

my_feature_names = ['Sex', 'Age','SibSp','Parch','Fare','is_S', 'is_C', 'is_Q' ]

dot_data = StringIO()
export_graphviz(clf_tree, feature_names=my_feature_names,
               out_file=dot_data, filled=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(value=graph.create_png())



Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x05\x12\x00\x00\x01\xf1\x08\x02\x00\x00\x00\x9e\\\xd…

Отметим нестабильность дерева. При изменении max_features=3 дерево заметно меняется (меняются важные признаки глубины 1).

### Самые важные признаки

Самый важный признак - пол пассажира. На данном узле энтропия уменьшается больше всего, данный узел выше всех.
Еще важынй признаки плата за билет и возраст.

В L1 регуляризации максимальные веса были также у пола и цены билета. ([-0.51781201(пол) -0.18299523 -0.20908921 -0.10937898  0.56521648(цена за билет) -0.05500351 0.07227682 -0.14884301])


### Случайной лес без кросс-валидацией с кросс-валидацией



In [61]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=10, max_depth=5).fit(X_train, y_train)

predicted_rf = rf.predict(X_test)

prec_score_rand_forest = precision_score(y_test, predicted_rf)

print('prec_score_rand_forest:', prec_score_rand_forest)

prec_score_rand_forest: 0.9312977099236641


In [68]:
# ищу смысл тут

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score



rf = RandomForestClassifier(n_estimators=10, max_depth=5)


#kf = KFold(n=len(X), n_folds=5, shuffle=True, random_state=42)
 
#kMeans = list()
#for k in range(1, 51):
#    kn = RandomForestClassifier(n_estimators=10, max_depth=5)
#    kn.fit(X_train, y_train);
#    array = cross_val_score(estimator=kn, X=X, y=y, cv=kf, scoring='accuracy')
#    m = array.mean()
#    kMeans.append(m)


#from sklearn.model_selection import cross_validate
#rf1=RandomForestRegressor(n_estimators=10)

#scores_val = cross_val_score(rf1, X_test, y_test, cv=5)
#print(scores_val)
#cv_results = cross_validate(rf1, X_test, y_test, cv=5)
#cv_results['test_score']

[1. 1. 1. 1. 1.]


array([1., 1., 1., 1., 1.])